***DBSCAN ( dimensity- Based Spaptial Clustering Of Application With Noise***

 DBSCAN adalah algprtima clustering yang:
 1. Mengelompokkan data berdasarkan kepadatan (Density)
 2. Dapat menemukan cluster dengan bentuk arbitrary ( tidak harus bulat )
3.  Dapat mengidentifikasi noise/outlier
4.  Tidak perlu menentukan jumlah cluster di awal ( berbeda dengan K-means )

Import Library

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from docutils.languages.af import labels
from sklearn.cluster import DBSCAN
from sklearn.datasets import make_moons, make_circles, make_blobs
from sklearn.preprocessing import StandardScaler

print("done")

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

FUNGSI UNTUK MEMBUAT DATA DAN MEMVISUALISASIKAN

In [ ]:
def generate_sample_data(data_type = 'moons'):
    """
    membuat data sampel untuk clustering

    Parameters :
    - data_type : tipe data yang akan dibuat
    * 'moons'   : data berbentuk bulan sabit ( 2 cluster melengkung)
    * 'circles' : data berbentuk lingkaran konsentris
    * 'blobs'   : data berbentuk blob/gumpalan terpisah
    * 'mixed'   : kombinasi dengan noise

    Returns :
    - X: array data 2D
    """
    if data_type == 'moons':
        #Data berbentuk 2 bulan sabit yang saling berhadapan
        X, _ = make_moons(n_samples=300, noise=0.05, random_state=42)

    elif data_type == 'circles':
        # Data berbentuk 2 lingkaran konsentris
        X, _ = make_circles(n_samples=300, noise=0.05, factor=0.5, random_state=42)

    elif data_type == 'blobs':
        # data berupa 3 gumpalan terpisah
        X, _ = make_blobs(n_samples=300, noise=0.05, cluster_std=0.5, random_state=42)

    elif data_type == 'mixed':
        # data kombinasi dengan noise untuk menunjukkan kemampuan DBSCAN
        X1, _ = make_blobs(n_samples=200, centers=2, cluster_std=0.4, random_state=42)
        X2 = np.random.uniform(-3,3, (50,2)) # noise
        X = np.vstack([X1, X2])

    return X




FUNGSI UNTUK MELAKUKAN DBSCAN CLUSTERING

In [ ]:
def perfrom_dbscan(X, eps=0.3, min_samples=5):
    """
    Melakukan clustering dengan DBSCAN

    Parameters Penting DBSCAN
    - eps(epsilon) : jarak maksimum antara 2 titik agar dapat di anggap tetangga
    * Nilai kecil : cluster lebih ketat, lebih banyak noise
    * Nilai besar : cluster lebih padat, lebih banyak noise

    - min samples : jumlah minimum titik dalam radius eps untuk membentuk cluster
    * Nilai kecil : kebih banyak cluster kecil
    * Nilai besar : cluster lebih padat, lebih banyak noise

    Returns :
    - Labels : array label cluster untuk setiap titik
                -1 berarti noise/outlier
                0,1,2,... adalah label cluster
    - n_clusters : jumlah cluster yang ditemukan
    -n_noise : jumlah titik yang dianggap noise
    """

    #PREPOCESSING : normalisasi data agar fitur memiliki skala yang sama
    # penting untuk dbscan karena mengguanakn jarak euclidean

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    #INISIALISASI MODEL DBSCAN
    # metric = 'euclidean' adalah default , menggunakan jarak euclidean

    dbscan = DBSCAN(eps=eps, min_samples=min_samples, metric='euclidean')

    # Fittung Model : menjalankan algoritma DBSCAN pada data
    labels = dbscan.fit_predict(X_scaled)

    #ANalisis hasil
    # label - 1 menunjukkan noise / outlier
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = list(labels).count(-1)

    print(f"\n {'n'*60}")
    print(f"HASIL CLUSTERING DBSCAN:")
    print(f"{'='*60}")
    print(f"Parameter eps (epsilon      : {eps}")
    print(f"Parameter min_samples       : {min_samples}")
    print(f"Parameter n_clusters        : {n_clusters}")
    print(f"Parameter n_noise           : {n_noise}")
    print(f"Jumlah Total titik data     : {len(labels)}")

    return labels, n_clusters, n_noise



FUNGSI UNUTK VISUALISASI HASIL CLUSTERING

In [ ]:
def visualize_clustering(X, lables, title="DBSCAN CLUSTERING"):
    """
    Mmevisualisasikan hasil clustering

    Parameters :
    X : data  original
    labels : hasil clustering dari DBSCAN
    title : Judul plot
    """

    # membuat figure plot
    plt.figure(fgsize=(10,6))

    #mendapatkan unique labels ( termasuk -1 untuk noise )
    unique_labels = set(labels)

    # membuat colormap untuk segitiga cluster
    # warna hitam #000000 untuk noise
    colors = plt.cm.Spectral(np.linspace(0,1, len(unique_labels)))

    # plot setiap cluster dengan warna berbeda
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Noise : warna hitam dengan marker 'x'
            col = [0,0,0,1]
            marker = 'x'
            label = 'Noise/Outlier'

            # memilih data points yang termasuk dalam cluster.noise
            class_member_mask = (labels == k)
            xy = X[class_member_mask]

            #plot point
            plt.scatter(xy[:, 0], xy[:,1],
                        c=[col], marker=marker,
                        edgecolors='black', linewidths=0,5,
                        label = label)
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel('Features 1 ', fontsize=12)
    plt.ylabel('Features 2 ', fontsize=12)
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()


FUNGSI EKSPERIMEN DENGAN PARAM BERBEDA

In [ ]:

def experiment_parameters(X):
    """
    Eksperimen dengn berbagai kombinasi parameter DBSCAN
    untuk memahami pengaruh eps dan nin samoles
    """

    eps_values = [0.2,0.3,0.5]
    min_samples_values = [3,5,10]

    fig, axes = plt.subplot(len(eps_values), len(min_samples_values),
                            figsize=(15,12))
    fig.suptitle('Eksperumen Parameter DBSCAN', fontsize=14, fontweight='bold')

    for i, eps in enumerate(eps_values):
        for j, min_samp in enumerate(min_samples_values):
            # perform clustering
            labels, n_clusters, n_noise = perfrom_dbscan(X, eps, min_samp)

            # normalisasi untuk visualisasi ( sama seperti dalam perform dbscan)
            scaler = StandardScaler()
            x_scaled = scaler.fit_transform(X)

            # plot pada subplot
            ax = axes[i,j]
            unique_labels = set(labels)
            colors = plt.cm.Spectral(np.linspace(0,1, len(unique_labels)))

            for k, col in zip(unique_labels, colors):
                if k == -1:
                    col = [0,0,0,1]
                    marker = 'x'
                else:
                    marker = 'o'

                class_member_mask = (labels == k)
                xy = X_scaled[class_member_mask]
                ax.scatter(xy[:, 0], xy[:,1], c=[col], marker=marker,
                           s=30, alpha=0.6, edgecolors='black', linewidths=0.5)

                ax.set_titile(f'eps={eps} min_samp={min_samp}\n' f'Cluster= {n_clusters}, n_noise={n_noise}', fontsize=14, fontweight='bold')
                ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()



In [ ]:
# ============================================================================
# FUNGSI UTAMA - MAIN PROGRAM
# ============================================================================

def main():
    """
    Fungsi utama untuk menjalankan program
    """

    print("\n" + "="*60)
    print("PROGRAM PEMBELAJARAN DBSCAN CLUSTERING")
    print("="*60)

    # ========================================================================
    # CONTOH 1: Data Berbentuk Bulan Sabit (Moons)
    # ========================================================================
    print("\n>>> CONTOH 1: Clustering pada Data Berbentuk Moons")
    print("Data ini memiliki 2 cluster berbentuk bulan sabit.")
    print("DBSCAN unggul di sini karena dapat mengenali bentuk non-linear!")

    X_moons = generate_sample_data('moons')
    labels_moons, _, _ = perform_dbscan(X_moons, eps=0.3, min_samples=5)
    visualize_clustering(X_moons, labels_moons,
                        "DBSCAN pada Data Moons (eps=0.3, min_samples=5)")

    # ========================================================================
    # CONTOH 2: Data Berbentuk Lingkaran Konsentris (Circles)
    # ========================================================================
    print("\n>>> CONTOH 2: Clustering pada Data Berbentuk Circles")
    print("Data ini memiliki 2 lingkaran konsentris.")
    print("K-Means akan gagal di sini, tapi DBSCAN bisa!")

    X_circles = generate_sample_data('circles')
    labels_circles, _, _ = perform_dbscan(X_circles, eps=0.2, min_samples=5)
    visualize_clustering(X_circles, labels_circles,
                        "DBSCAN pada Data Circles (eps=0.2, min_samples=5)")

    # ========================================================================
    # CONTOH 3: Data dengan Noise
    # ========================================================================
    print("\n>>> CONTOH 3: Clustering pada Data dengan Noise")
    print("Data ini memiliki cluster plus banyak noise.")
    print("DBSCAN dapat mengidentifikasi outlier dengan baik!")

    X_mixed = generate_sample_data('mixed')
    labels_mixed, _, _ = perform_dbscan(X_mixed, eps=0.3, min_samples=5)
    visualize_clustering(X_mixed, labels_mixed,
                        "DBSCAN pada Data dengan Noise (eps=0.3, min_samples=5)")

    # ========================================================================
    # CONTOH 4: Eksperimen Parameter
    # ========================================================================
    print("\n>>> CONTOH 4: Eksperimen dengan Berbagai Parameter")
    print("Mari kita lihat bagaimana parameter eps dan min_samples")
    print("mempengaruhi hasil clustering...")

    experiment_parameters(X_moons)

    # ========================================================================
    # KESIMPULAN DAN TIPS
    # ========================================================================
    print("\n" + "="*60)
    print("KESIMPULAN DAN TIPS MENGGUNAKAN DBSCAN:")
    print("="*60)
    print("""
1. KELEBIHAN DBSCAN:
   ✓ Tidak perlu menentukan jumlah cluster di awal
   ✓ Dapat menemukan cluster dengan bentuk arbitrary (tidak harus bulat)
   ✓ Dapat mengidentifikasi noise/outlier
   ✓ Robust terhadap outlier

2. KEKURANGAN DBSCAN:
   ✗ Sensitif terhadap parameter eps dan min_samples
   ✗ Kurang baik untuk data dengan density yang sangat berbeda
   ✗ Tidak baik untuk data berdimensi tinggi (curse of dimensionality)

3. TIPS MEMILIH PARAMETER:
   - eps: Gunakan K-distance graph untuk menentukan nilai optimal
   - min_samples: Aturan umum: min_samples = dimensi_data + 1
                  Untuk 2D: minimal 3, biasanya 5-10

4. KAPAN MENGGUNAKAN DBSCAN:
   ✓ Data memiliki cluster dengan bentuk tidak beraturan
   ✓ Ada banyak noise dalam data
   ✓ Tidak tahu berapa jumlah cluster yang tepat
   ✓ Density antar cluster relatif seragam
    """)
    print("="*60 + "\n")

# ============================================================================
# JALANKAN PROGRAM
# ============================================================================
if __name__ == "__main__":
    main()

    # LATIHAN TAMBAHAN:
    # Uncomment baris di bawah untuk mencoba data Anda sendiri!
    """
    # Contoh membuat data custom
    my_data = np.random.randn(100, 2)  # 100 titik, 2 fitur
    my_labels, _, _ = perform_dbscan(my_data, eps=0.5, min_samples=5)
    visualize_clustering(my_data, my_labels, "Clustering Data Custom")
    """